<a href="https://colab.research.google.com/github/JandreiSS/Analise-Fundamentalista/blob/master/DataFrame_bolsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import requests
import warnings
warnings.filterwarnings('ignore')
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [2]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [3]:
url = 'http://fundamentus.com.br/resultado.php'

In [4]:
r = requests.get(url, headers=header)

In [5]:
df = pd.read_html(r.text)[0]

In [6]:
df = df.loc[:,['Papel']]

In [7]:
def get(url, timeout):
    return requests.get(url, headers=header, timeout = timeout)

In [8]:
def requestUrls(urls, timeout = 5):
    with ThreadPoolExecutor(max_workers = 3) as executor:
        agenda = { executor.submit(get, url, timeout): url for url in urls }

        for tarefa in as_completed(agenda):     
            try:
                conteudo = tarefa.result()
            except Exception as e:
                print ("\nNão foi possível fazer a requisição! \n{}".format(e))
            else:
                yield conteudo

In [9]:
urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + df['Papel'].values

In [10]:
requisicoes = requestUrls(urls, timeout=60)

In [11]:
lista_empresas = []

In [12]:
total_ticket = len(df)

In [13]:
for requisicao in tqdm(requisicoes, unit=' requisições', desc='Requisitando dados: ', disable=False, total=total_ticket, ncols=100):
    # codigo = requisicao.status_code
    # url = requisicao.url
    conteudo = requisicao.content
    lista_empresas += [conteudo]
    # print (f'{codigo}: {url}')

Requisitando dados: 100%|███████████████████████████████| 887/887 [05:22<00:00,  2.75 requisições/s]


### Transformar os dados do HTML para um DataFrame

In [14]:
colunas = [
    'Papel',
    'Empresa',
    'Setor',
    'Subsetor',
    'Cotação',
    'Valor da Firma',
    'Valor de Mercado',
    'Número de Ações',
    'P/L',
    'Div. Yield',
    'EV/EBITDA',
    'EV/EBIT',
    'ROE'
]

In [15]:
tabela_base = pd.DataFrame(columns=colunas)

In [16]:
total_lista = len(lista_empresas)

In [17]:
for key in tqdm(range(total_lista), unit=' dados', desc='Tratando dados: ', disable=False, total=total_lista, ncols=100):
    dados_base = pd.read_html(lista_empresas[key])
    papel = dados_base[0][1][0]
    empresa = dados_base[0][1][2]
    setor = dados_base[0][1][3]
    subsetor = dados_base[0][1][4]
    cotacao = dados_base[0][3][0]
    valor_da_firma = dados_base[1][1][1]
    valor_de_mercado = dados_base[1][1][0]
    n_acoes = dados_base[1][3][1]
    p_l = dados_base[2][3][1]
    div_yield = dados_base[2][3][8]
    ev_ebitda = dados_base[2][3][9]
    ev_ebit = dados_base[2][3][10]
    roe = dados_base[2][5][8]
    to_append = [papel, empresa, setor, subsetor, cotacao, valor_da_firma, valor_de_mercado, n_acoes, p_l, div_yield, ev_ebitda, ev_ebit, roe]
    tabela_base_length = len(tabela_base)
    tabela_base.loc[tabela_base_length] = to_append


Tratando dados: 100%|█████████████████████████████████████████| 887/887 [00:54<00:00, 16.39 dados/s]


### Tratando colunas com %

In [20]:
for coluna in ['ROE', 'Div. Yield']:
    tabela_base[coluna] = tabela_base[coluna].str.rstrip('%')

In [21]:
tabela_base

,Papel,Empresa,Setor,Subsetor,Cotação,Valor da Firma,Valor de Mercado,Número de Ações,P/L,Div. Yield,EV/EBITDA,EV/EBIT,ROE
0,CFLU4,COCA COLA PN,NaN,NaN,1.00000,-,0,0,000,"0,0",-,-,"32,2"
1,CLAN3,CLARION ON,Alimentos,Grãos e Derivados,000,-,0,158.703.000,000,"0,0",-,-,"-1,0"
2,VNET3,CIELO ON NM,Serviços Financeiros Diversos,Serviços Financeiros Diversos,000,-,0,2.716.820.000,000,"0,0",-,-,"13,0"
3,CLSC6,CELESC PNB N2,Energia Elétrica,Energia Elétrica,000,-,0,38.572.000,000,"0,0",-,-,"22,9"
4,CSTB4,COMPANHIA SIDERÚRGICA DE TUBARÃO PN,NaN,NaN,14769,-,0,0,000,"0,0",-,-,"20,1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,UBBR3,UNIBANCO SA ON N1,Financeiros,Bancos,1800,-,49.219.500.000,2.734.420.000,1.46661,"0,0",-,-,"0,3"
883,CEPE3,CELPE ON,Energia Elétrica,Energia Elétrica,2453,6.818.230.000,1.830.230.000,74.612.000,1.89599,"0,0",26996,32616,"0,1"
884,CEPE5,CELPE PNA,Energia Elétrica,Energia Elétrica,2700,7.002.520.000,2.014.520.000,74.612.000,2.08690,"0,0",27726,33498,"0,1"
885,BRAP3,BRADESPAR S/A ON N1,Holdings Diversificadas,Holdings Diversificadas,3678,12.826.500.000,12.800.700.000,348.034.000,2.26963,"2,0",-73437,-73437,"0,1"


In [22]:
tabela_base.to_csv('tabela_dados.csv', index=False, sep=';')